In [1]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Dash config with parent asset path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🐟 Fish Carousel"

# 📦 Liste des espèces
FISH_SPECIES = sorted([
    d for d in os.listdir("../assets/fish")
    if os.path.isdir(f"../assets/fish/{d}")
])

# 📦 Dictionnaire {species: [image paths]}
fish_images = {}
for species in FISH_SPECIES:
    folder = f"../assets/fish/{species}"
    images = []
    for root, _, files in os.walk(folder):
        images += [
            os.path.abspath(os.path.join(root, f))
            for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
    fish_images[species] = sorted(images)

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🐟 Fish Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select fish species:", style={
            "marginBottom": "6px", "frontSize": "15px", "frontStyle": "normal"
            
        }),
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.capitalize(), "value": s} for s in FISH_SPECIES],
            value=FISH_SPECIES[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),


    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Affiche carousel
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = fish_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))
    dash_path = f"/assets/{relative_path}"

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "20px", "padding": "6px 12px", "marginRight": "20px"}),

            html.Img(id="carousel-image", src=dash_path,
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),

            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "20px", "padding": "6px 12px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "15px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),

        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "13px", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8807)
